# Inference examples

In [ ]:
# Do not forget to install all dependencies first:
!pip install -Uqq WhisperSpeech

In [ ]:
def is_colab():
    try: import google.colab; return True
    except: return False

import torch
if not torch.cuda.is_available():
    if is_colab(): raise BaseException("Please change the runtime type to GPU. In the menu: Runtime -> Change runtime type (the free T4 instance is enough)")
    else:          raise BaseException("Currently the example notebook requires CUDA, make sure you are running this on a machine with a GPU.")

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn.functional as F

from IPython.display import Markdown, HTML

## The whole pipeline

In [ ]:
# check "7. Pipeline.ipynb"
from whisperspeech.pipeline import Pipeline

In [ ]:
# let's start with the fast SD S2A model
pipe = Pipeline(s2a_ref='collabora/whisperspeech:s2a-q4-tiny-en+pl.model')

/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
# this is very slow right now since our inference code is not very optimized
# but even without this crucial optimization it is still better than real-time on an RTX 4090
pipe.generate_to_notebook("""
This is the first demo of Whisper Speech, a fully open source text-to-speech model trained by Collabora and Lion on the Juwels supercomputer.
""")

In [ ]:
# The model knows how to speak in Polish
pipe.generate_to_notebook("""
To jest pierwszy test naszego modelu. Pozdrawiamy serdecznie.
""", lang='pl')

In [ ]:
# We can also mix different languages (e.g. for borrowed words) in a single sentence
stoks = pipe.t2s.generate(["To jest pierwszy test wielojęzycznego ", " Whisper Speech ", ", modelu zamieniającego tekst na mowę, który Collabora i Laion nauczyli na superkomputerze", " Jewels."], lang=['pl', 'en', 'pl', 'en'])
stoks = stoks[stoks != 512]
pipe.vocoder.decode_to_notebook(pipe.s2a.generate(stoks, pipe.default_speaker.unsqueeze(0)))

In [ ]:
stoks = pipe.t2s.generate(["I love to eat eastern european food! Especially ", "pierogi i bigos."], lang=['en', 'pl'], cps=11)
stoks = stoks[stoks != 512]
pipe.vocoder.decode_to_notebook(pipe.s2a.generate(stoks, pipe.default_speaker.unsqueeze(0)))

## Voice cloning

In [ ]:
# we can give it an audio file reference to get zero-shot voice cloning
#
# you can provide a URL or upload your own audio files
pipe.generate_to_notebook("""
This is the first demo of Whisper Speech, a fully open source text-to-speech model trained by Collabora and Lion on the Juwels supercomputer.
""", lang='en', cps=10, speaker='https://upload.wikimedia.org/wikipedia/commons/7/75/Winston_Churchill_-_Be_Ye_Men_of_Valour.ogg')

/opt/conda/lib/python3.10/site-packages/torch/functional.py:660: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:874.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]


In [ ]:
# it works even better (but slower) with the HQ model
pipe = Pipeline()

/opt/conda/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [ ]:
pipe.generate_to_notebook("""
This is the first demo of Whisper Speech, a fully open source text-to-speech model trained by Collabora and Lion on the Juwels supercomputer.
""", lang='en', cps=10, speaker='https://upload.wikimedia.org/wikipedia/commons/7/75/Winston_Churchill_-_Be_Ye_Men_of_Valour.ogg')